<a href="https://colab.research.google.com/github/kitae0522/my_data_set/blob/main/PistolDetect.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!curl -L "https://public.roboflow.com/ds/fFy2xusWLi?key=lPLerhz8wp" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip

In [ ]:
%cd /content
!git clone https://github.com/ultralytics/yolov5.git

/content
Cloning into 'yolov5'...
remote: Enumerating objects: 15, done.
remote: Counting objects: 100% (15/15), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 3106 (delta 4), reused 2 (delta 1), pack-reused 3091
Receiving objects: 100% (3106/3106), 6.73 MiB | 6.52 MiB/s, done.
Resolving deltas: 100% (2060/2060), done.


In [ ]:
%cd /content/yolov5/
!pip install -r requirements.txt

/content/yolov5
     |████████████████████████████████| 276kB 13.1MB/s 
  Created wheel for PyYAML: filename=PyYAML-5.3.1-cp36-cp36m-linux_x86_64.whl size=44619 sha256=f47740c7e8ac0452a729893e4c02e86507c606bd3339dc981cc3447319258893
  Stored in directory: /root/.cache/pip/wheels/a7/c1/ea/cf5bd31012e735dc1dfea3131a2d5eae7978b251083d6247bd
Successfully built PyYAML
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
%cat /content/DataSet/data.yaml

train: ../train/images
val: ../valid/images

nc: 1
names: ['pistol']

In [ ]:
%cd /
from glob import glob

img_list = glob("/content/DataSet/export/images/*.jpg")
print(len(img_list))

/
2971


In [ ]:
from sklearn.model_selection import train_test_split
train_img_list, val_img_list = train_test_split(img_list, test_size=0.2, random_state=2000)
print(len(train_img_list), len(val_img_list))

2376 595


In [ ]:
with open("content/DataSet/train.txt", "w") as f:
  f.write("\n".join(train_img_list)+"\n")

with open("content/DataSet/val.txt", "w") as f:
  f.write("\n".join(val_img_list)+"\n")

In [ ]:
import yaml

with open("content/DataSet/data.yaml", "r") as f:
    data = yaml.load(f)

print(data)

data["train"]= "/content/DataSet/train.txt"
data["val"]= "/content/DataSet/val.txt"

with open('/content/DataSet/data.yaml', 'w') as f:
  yaml.dump(data, f)

print(data)

{'train': '../train/images', 'val': '../valid/images', 'nc': 1, 'names': ['pistol']}
{'train': '/content/DataSet/train.txt', 'val': '/content/DataSet/val.txt', 'nc': 1, 'names': ['pistol']}


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  after removing the cwd from sys.path.


In [ ]:
%cd /content/yolov5/

!python train.py --img 416 --batch 16 --epochs 200 --data /content/DataSet/data.yaml --cfg ./models/yolov5s.yaml --weights yolov5s.pt --name gun_yolov5s_results

/content/yolov5
Using torch 1.7.0+cu101 CUDA:0 (Tesla P100-PCIE-16GB, 16280MB)

Namespace(adam=False, batch_size=16, bucket='', cache_images=False, cfg='./models/yolov5s.yaml', data='/content/DataSet/data.yaml', device='', epochs=200, evolve=False, exist_ok=False, global_rank=-1, hyp='data/hyp.scratch.yaml', image_weights=False, img_size=[416, 416], local_rank=-1, log_imgs=16, multi_scale=False, name='gun_yolov5s_results', noautoanchor=False, nosave=False, notest=False, project='runs/train', rect=False, resume=False, save_dir='runs/train/gun_yolov5s_results6', single_cls=False, sync_bn=False, total_batch_size=16, weights='yolov5s.pt', workers=8, world_size=1)
Start Tensorboard with "tensorboard --logdir runs/train", view at http://localhost:6006/
2020-11-26 11:38:28.493247: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
Hyperparameters {'lr0': 0.01, 'lrf': 0.2, 'momentum': 0.937, 'weight_decay': 0.0005, 'warmup_epoc

In [ ]:
#영상에서 pistol 찾아내기

import torch
from IPython.display import Image, clear_output  # to display images

clear_output()
print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

Setup complete. Using torch 1.7.0+cu101 _CudaDeviceProperties(name='Tesla P100-PCIE-16GB', major=6, minor=0, total_memory=16280MB, multi_processor_count=56)


In [ ]:
from IPython.display import Image
import os

val_img_path = val_img_list[0]

!python detect.py --weights /content/yolov5/runs/train/gun_yolov5s_results/weights/best.pt --img 416 --conf 0.5 --source "{val_img_path}"

Namespace(agnostic_nms=False, augment=False, classes=None, conf_thres=0.5, device='', exist_ok=False, img_size=416, iou_thres=0.45, name='exp', project='runs/detect', save_conf=False, save_txt=False, source='/content/DataSet/export/images/armas (1886)_jpg.rf.ca8db38ff01fcb45220e7d6dbc5b02c1.jpg', update=False, view_img=False, weights=['/content/yolov5/runs/train/gun_yolov5s_results/weights/best.pt'])
Using torch 1.7.0+cu101 CUDA:0 (Tesla P100-PCIE-16GB, 16280MB)

Fusing layers... 
Model Summary: 232 layers, 7246518 parameters, 0 gradients
image 1/1 /content/DataSet/export/images/armas (1886)_jpg.rf.ca8db38ff01fcb45220e7d6dbc5b02c1.jpg: 416x416 Done. (0.010s)
Results saved to runs/detect/exp7
Done. (0.027s)


In [ ]:
!python detect.py --weights /content/yolov5/runs/train/gun_yolov5s_results/weights/best.pt --source /content/DataSet/pistol.mp4

Namespace(agnostic_nms=False, augment=False, classes=None, conf_thres=0.25, device='', exist_ok=False, img_size=640, iou_thres=0.45, name='exp', project='runs/detect', save_conf=False, save_txt=False, source='/content/DataSet/pistol.mp4', update=False, view_img=False, weights=['/content/yolov5/runs/train/gun_yolov5s_results/weights/best.pt'])
Using torch 1.7.0+cu101 CUDA:0 (Tesla P100-PCIE-16GB, 16280MB)

Fusing layers... 
Model Summary: 232 layers, 7246518 parameters, 0 gradients
video 1/1 (1/2498) /content/DataSet/pistol.mp4: 384x640 Done. (0.011s)
video 1/1 (2/2498) /content/DataSet/pistol.mp4: 384x640 Done. (0.010s)
video 1/1 (3/2498) /content/DataSet/pistol.mp4: 384x640 Done. (0.012s)
video 1/1 (4/2498) /content/DataSet/pistol.mp4: 384x640 Done. (0.010s)
video 1/1 (5/2498) /content/DataSet/pistol.mp4: 384x640 Done. (0.010s)
video 1/1 (6/2498) /content/DataSet/pistol.mp4: 384x640 Done. (0.010s)
video 1/1 (7/2498) /content/DataSet/pistol.mp4: 384x640 Done. (0.010s)
video 1/1 (8/2498